## Environment

In [2]:
# run in cellpose_2 conda environment (anaconda terminal) in the alien computer
import os
import time
import pathlib

import numpy as np
import pandas as pd

import tifffile
from skimage.morphology import white_tophat, black_tophat, disk
from skimage import measure, util, morphology, segmentation

from scipy import ndimage
from scipy.sparse import csr_matrix

from cellpose import models
from tqdm import tqdm

## Read image

In [3]:
# read image
file_name = "/Volumes/Jeff-exFAT/For_Ash/23-09-13/ISC/1/ISC-12d-ph3-01.tif"
img = tifffile.imread(file_name)
img_name = pathlib.Path(file_name).stem

print(img_name)
print(img.shape)
print(img.dtype)

ISC-12d-ph3-01
(14, 3, 512, 512)
uint16


## Get nuclear masks
 - Nuclear mask --> for nuclear signal intensities
 - Dilated nuclear mask --> for cytoplasmic signal intensities

In [6]:
# - - - - - Nuclear mask
## get segmentation channel
seg_channel = 2
seg_img = img[:, seg_channel, :, :]

## median filter
median_filter_size = 1
seg_img_median = []
for z in seg_img:
    seg_img_median_slice = ndimage.median_filter(z, size = median_filter_size)
    seg_img_median.append(seg_img_median_slice)
seg_img = np.array(seg_img_median)

## cellpose parameters
diameter = 18
flow_threshold = 0
mask_threshold = -5
do_3D = True
stitch_threshold = 0
gpu = False

## run cellpose in 3D mode
model = models.Cellpose(gpu = gpu, model_type = 'cyto')
masks, flows, styles, diams = model.eval(
    seg_img, 
    channels = [0,0], 
    diameter = diameter, 
    do_3D = do_3D, 
    flow_threshold = flow_threshold, 
    cellprob_threshold = mask_threshold
    )

# - - - - - Dilated nuclear mask 

# ## helper functions
# def dilate_mask_cupy(segmentation_array, label_id, dilation_iterations):
#     current_label_id = cupy.where(cupy.array(segmentation_array) == label_id, 1, 0)
#     dilated = cupyx.scipy.ndimage.binary_dilation(cupy.array(current_label_id), iterations = dilation_iterations)
#     relabeled_dilated = cupy.where(dilated == 1, label_id, 0)
#     return(np.array(relabeled_dilated))

# def dilate_labels_cupy(segmentation, dilation_iterations):
#     list_of_dilated_masks = list()
#     regions = measure.regionprops(segmentation)
#     for i in range(len(regions)):
#         label_id = regions[i].label
#         list_of_dilated_masks.append(dilate_mask_cupy(segmentation, label_id, dilation_iterations))
#     final_array_labelled = sum(list_of_dilated_masks)
#     return(final_array_labelled)

## get expanded masks of each cell labels 
expansion_px = 2
masks_expanded = []
for z in masks:
    masks_expanded_slice = segmentation.expand_labels(z, distance = expansion_px)
    masks_expanded.append(masks_expanded_slice)
masks_expanded = np.array(masks_expanded)

## get doughnut mask
masks_doughnut = masks_expanded - masks

# - - - - - Save mask images

## save mask image
output_dir = "/Users/jefflee/Desktop/img_test/"
masks_path = pathlib.Path(output_dir).joinpath(f"{img_name}_masks.tiff")
masks_expanded_path = pathlib.Path(output_dir).joinpath(f"{img_name}_masks_expanded.tiff")
masks_doughnut_path = pathlib.Path(output_dir).joinpath(f"{img_name}_masks_doughnut.tiff")

tifffile.imwrite(masks_path, masks)
tifffile.imwrite(masks_expanded_path, masks_expanded)
tifffile.imwrite(masks_doughnut_path, masks_doughnut)


## Get nuclei property details

In [31]:
nuc_regionprops = measure.regionprops_table(
    label_image = masks,
    properties = ("label", "area", "centroid", "axis_major_length", "axis_minor_length")
)

nuc_df = pd.DataFrame(nuc_regionprops)
nuc_df

,label,area,centroid-0,centroid-1,centroid-2,axis_major_length,axis_minor_length
0,1,386.0,1.297927,164.694301,20.056995,24.995510,2.967763
1,2,36.0,0.000000,163.666667,22.277778,10.458072,0.000000
2,3,206.0,4.174757,177.723301,150.432039,12.370558,5.543911
3,4,427.0,2.482436,181.159251,154.969555,16.437339,5.294031
4,5,534.0,1.104869,200.481273,227.119850,17.756787,3.827936
...,...,...,...,...,...,...,...
520,521,412.0,15.152913,189.922330,423.250000,18.227264,3.599806
521,522,64.0,15.343750,243.843750,485.125000,7.737315,3.199501
522,523,112.0,15.276786,416.089286,32.687500,10.195592,3.518141
523,524,41.0,15.268293,446.317073,150.756098,17.259956,2.056487


## Calculate fluorescence intensities

In [33]:
# - - - - - esg > GFP intensities

## background subtraction helper function
def subtract_background(image, radius, light_bg=False):
    # you can also use 'ball' here to get a slightly smoother result at the
    # cost of increased computing time
    str_el = disk(radius)
    if light_bg:
        return black_tophat(image, str_el)
    else:
        return white_tophat(image, str_el)
    
## Subtract background
GFP_channel = 0
bgs_radius = 20

GFP_img = img[:, GFP_channel, :, :]

GFP_bgs = []
for z in GFP_img:
    GFP_bgs_slice = subtract_background(z, bgs_radius)
    GFP_bgs.append(GFP_bgs_slice)
GFP_img = np.array(GFP_bgs)

## Measure GFP fluorescence intensity against doughnut mask
GFP_regionprops = measure.regionprops_table(
    label_image = masks_doughnut,
    intensity_image = GFP_img,
    properties = ("label", "area", "image_intensity")
)

GFP_df = pd.DataFrame(GFP_regionprops)
GFP_df["GFP_intensity_sum"] = GFP_df["image_intensity"].transform(lambda x: np.sum(x))
GFP_df = GFP_df[["label", "area", "GFP_intensity_sum"]]
GFP_df = GFP_df.rename(columns={"area":"doughnut_area"})

GFP_df

,label,doughnut_area,GFP_intensity_sum
0,1,336.0,590488
1,2,16.0,41878
2,3,242.0,3550993
3,4,346.0,1732876
4,5,320.0,456054
...,...,...,...
520,521,219.0,3549260
521,522,115.0,1426506
522,523,153.0,3109099
523,524,119.0,2392979


## Combine nuclear + GFP dataframes

In [36]:
output_df = nuc_df.merge(GFP_df, on = "label", how = "left")
output_df

,label,area,centroid-0,centroid-1,centroid-2,axis_major_length,axis_minor_length,doughnut_area,GFP_intensity_sum
0,1,386.0,1.297927,164.694301,20.056995,24.995510,2.967763,336.0,590488
1,2,36.0,0.000000,163.666667,22.277778,10.458072,0.000000,16.0,41878
2,3,206.0,4.174757,177.723301,150.432039,12.370558,5.543911,242.0,3550993
3,4,427.0,2.482436,181.159251,154.969555,16.437339,5.294031,346.0,1732876
4,5,534.0,1.104869,200.481273,227.119850,17.756787,3.827936,320.0,456054
...,...,...,...,...,...,...,...,...,...
520,521,412.0,15.152913,189.922330,423.250000,18.227264,3.599806,219.0,3549260
521,522,64.0,15.343750,243.843750,485.125000,7.737315,3.199501,115.0,1426506
522,523,112.0,15.276786,416.089286,32.687500,10.195592,3.518141,153.0,3109099
523,524,41.0,15.268293,446.317073,150.756098,17.259956,2.056487,119.0,2392979


In [40]:
output_df.to_csv("./df.csv", output_df)

TypeError: "delimiter" must be string, not DataFrame